## Training

# Example Predictor: Linear Rollout Predictor

This example contains basic functionality for training and evaluating a linear predictor that rolls out predictions day-by-day.

First, a training data set is created from historical case and npi data.

Second, a linear model is trained to predict future cases from prior case data along with prior and future npi data.
The model is an off-the-shelf sklearn Lasso model, that uses a positive weight constraint to enforce the assumption that increased npis has a negative correlation with future cases.

Third, a sample evaluation set is created, and the predictor is applied to this evaluation set to produce prediction results in the correct format.

In [24]:
import pickle
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

### Copy the data locally

In [25]:
# Main source for the training data
# DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
# Local file
DATA_FILE = 'data/OxCGRT_latest.csv'

In [26]:
import os
import urllib.request
if not os.path.exists('data'):
    os.mkdir('data')
# urllib.request.urlretrieve(DATA_URL, DATA_FILE)

In [27]:
# Load historical data from local file
df = pd.read_csv(DATA_FILE, 
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)

In [28]:
df.columns

Index(['CountryName', 'CountryCode', 'RegionName', 'RegionCode',
       'Jurisdiction', 'Date', 'C1_School closing', 'C1_Flag',
       'C2_Workplace closing', 'C2_Flag', 'C3_Cancel public events', 'C3_Flag',
       'C4_Restrictions on gatherings', 'C4_Flag', 'C5_Close public transport',
       'C5_Flag', 'C6_Stay at home requirements', 'C6_Flag',
       'C7_Restrictions on internal movement', 'C7_Flag',
       'C8_International travel controls', 'E1_Income support', 'E1_Flag',
       'E2_Debt/contract relief', 'E3_Fiscal measures',
       'E4_International support', 'H1_Public information campaigns',
       'H1_Flag', 'H2_Testing policy', 'H3_Contact tracing',
       'H4_Emergency investment in healthcare', 'H5_Investment in vaccines',
       'H6_Facial Coverings', 'H6_Flag', 'M1_Wildcard', 'ConfirmedCases',
       'ConfirmedDeaths', 'StringencyIndex', 'StringencyIndexForDisplay',
       'StringencyLegacyIndex', 'StringencyLegacyIndexForDisplay',
       'GovernmentResponseIndex', 'Gove

In [29]:
# For testing, restrict training data to that before a hypothetical predictor submission date
HYPOTHETICAL_SUBMISSION_DATE = np.datetime64("2020-11-25")
df = df[df.Date <= HYPOTHETICAL_SUBMISSION_DATE]

In [30]:
# Add RegionID column that combines CountryName and RegionName for easier manipulation of data
# GeoID 作为地区的唯一识别号，用于后续的分组中
df['GeoID'] = df['CountryName'] + '__' + df['RegionName'].astype(str)

In [31]:
# Add new cases column
# ConfirmedCases 确诊病例  NewCases 新增病例
df['NewCases'] = df.groupby('GeoID').ConfirmedCases.diff().fillna(0)

In [32]:
# Keep only columns of interest
id_cols = ['CountryName',
           'RegionName',
           'GeoID',
           'Date']
cases_col = ['NewCases']
npi_cols = ['C1_School closing',
            'C2_Workplace closing',
            'C3_Cancel public events',
            'C4_Restrictions on gatherings',
            'C5_Close public transport',
            'C6_Stay at home requirements',
            'C7_Restrictions on internal movement',
            'C8_International travel controls',
            'H1_Public information campaigns',
            'H2_Testing policy',
            'H3_Contact tracing',
            'H6_Facial Coverings']
# 12 列为干预措施 
print(len(npi_cols))
df = df[id_cols + cases_col + npi_cols]

12


In [33]:
# Fill any missing case values by interpolation and setting NaNs to 0
df.update(df.groupby('GeoID').NewCases.apply(
    lambda group: group.interpolate()).fillna(0))

In [34]:
# Fill any missing NPIs by assuming they are the same as previous day
for npi_col in npi_cols:
    df.update(df.groupby('GeoID')[npi_col].ffill().fillna(0))

In [35]:
df

,CountryName,RegionName,GeoID,Date,NewCases,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings
0,Aruba,NaN,Aruba__nan,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,NaN,Aruba__nan,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aruba,NaN,Aruba__nan,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aruba,NaN,Aruba__nan,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aruba,NaN,Aruba__nan,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87973,Zimbabwe,NaN,Zimbabwe__nan,2020-11-21,74.0,1.0,1.0,2.0,3.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,3.0
87974,Zimbabwe,NaN,Zimbabwe__nan,2020-11-22,52.0,1.0,1.0,2.0,3.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,3.0
87975,Zimbabwe,NaN,Zimbabwe__nan,2020-11-23,48.0,1.0,1.0,2.0,3.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,3.0
87976,Zimbabwe,NaN,Zimbabwe__nan,2020-11-24,88.0,1.0,1.0,2.0,3.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,3.0


In [36]:
# Set number of past days to use to make predictions
# 回看的天数，可以自行设置
nb_lookback_days = 14

# Create training data across all countries for predicting one day ahead
# 用过去一个月的数据预测当天的值
X_cols = cases_col + npi_cols
y_col = cases_col


X_train_samples = dict()
X_test_samples = dict()

y_train_samples = dict()
y_test_samples = dict()

holdout_num = 14

geo_ids = df.GeoID.unique()
for g in geo_ids:
    X_samples = []
    y_samples = []
    
    # 筛选出特定区域的数据
    gdf = df[df.GeoID == g].copy()
    # 使用新增病例增长率来预测
    gdf['NewCasesRatio'] = gdf.NewCases.pct_change(
        ).fillna(0).replace(np.inf, 0) + 1
    
    all_case_data = np.array(gdf[cases_col])
    all_npi_data = np.array(gdf[npi_cols])
    
    # Create one sample for each day where we have enough data
    # Each sample consists of cases and npis for previous nb_lookback_days
    # 每个样本包括过去一个月病例和干预措施，相当于当天的预测只参考于过去一个月的数据， K阶马尔可夫
    nb_total_days = len(gdf)
    for d in range(nb_lookback_days, nb_total_days):
        X_cases = all_case_data[d-nb_lookback_days:d]
        # X_cases = all_case_data[d-1:d]
        # Take negative of npis to support positive
        # weight constraint in Lasso.
        # 取负值，是为了让权重变成正的
        X_npis = -all_npi_data[d - nb_lookback_days:d]
        # X_cases 是一个矩阵，需要拉平为一个向量
        # Flatten all input data so it fits Lasso input format.
        X_sample = np.concatenate([X_cases.flatten(),
                                   X_npis.flatten()])
        y_sample = all_case_data[d]
        X_samples.append(X_sample)
        y_samples.append(y_sample)
    
    X_train_samples[g] = X_samples[:-holdout_num]
    X_test_samples[g] = X_samples[-holdout_num:]
    
    y_train_samples[g] = y_samples[:-holdout_num]
    y_test_samples[g] = y_samples[-holdout_num:]
        
X_train = []
y_train =  [] 
X_test = []
y_test =  [] 

for geo, val in X_train_samples.items():
    #if geo.startswith('United States'):
    X_train += val
    y_train += y_train_samples[geo]
    

for geo, val in X_test_samples.items():
    X_test += val
    y_test += y_test_samples[geo]
    
X_train = np.array(X_train)
y_train = np.array(y_train).flatten()

X_test = np.array(X_train)
y_test = np.array(y_train).flatten()

#30 * 1 + 30 *12 前面30列为过去每天的新增病例 后面360列为过去30天的干预措施逐天叠加在一起
print(X_train, X_train.shape)
print(y_train.shape)

[[ 0.  0.  0. ... -0. -0. -0.]
 [ 0.  0.  0. ... -0. -0. -0.]
 [ 0.  0.  0. ... -0. -0. -0.]
 ...
 [ 0.  0.  5. ... -0. -0. -0.]
 [ 0.  5.  0. ... -0. -0. -0.]
 [ 5.  0.  0. ... -0. -0. -0.]] (16308, 182)
(16308,)


In [37]:
y_test_samples.keys()
def get_geo_samples(geo,X, y):
    return np.array(X[geo]), np.array(y[geo])
X_geo_test, y_geo_test = get_geo_samples('United States__New York', X_test_samples, y_test_samples)
print(y_geo_test)
df[df['GeoID'] == 'United States__New York'].tail()

[[4797.]
 [5401.]
 [5388.]
 [3649.]
 [3490.]
 [5088.]
 [5294.]
 [5310.]
 [5468.]
 [5973.]
 [5391.]
 [5906.]
 [4881.]
 [6265.]]


,CountryName,RegionName,GeoID,Date,NewCases,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings
79341,United States,New York,United States__New York,2020-11-21,5973.0,2.0,3.0,2.0,4.0,1.0,1.0,2.0,3.0,2.0,3.0,2.0,3.0
79342,United States,New York,United States__New York,2020-11-22,5391.0,2.0,3.0,2.0,4.0,1.0,1.0,2.0,3.0,2.0,3.0,2.0,3.0
79343,United States,New York,United States__New York,2020-11-23,5906.0,2.0,3.0,2.0,4.0,1.0,1.0,2.0,3.0,2.0,3.0,2.0,3.0
79344,United States,New York,United States__New York,2020-11-24,4881.0,2.0,3.0,2.0,4.0,1.0,1.0,2.0,3.0,2.0,3.0,2.0,3.0
79345,United States,New York,United States__New York,2020-11-25,6265.0,2.0,3.0,2.0,4.0,1.0,1.0,2.0,3.0,2.0,3.0,2.0,3.0


In [38]:
# Helpful function to compute mae
def mae(pred, true):
    return np.mean(np.abs(pred - true))

In [39]:
# Split data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(X_samples,
#                                                     y_samples,
#                                                     test_size=0.2,
#                                                     random_state=301)

In [40]:
# Create and train Lasso model.
# Set positive=True to enforce assumption that cases are positively correlated
# with future cases and npis are negatively correlated.
from sklearn.ensemble import BaggingRegressor
base_model = Lasso(alpha=0.1,
              precompute=True,
              max_iter=10000,
              positive=True,
              selection='random')
model = BaggingRegressor(base_model, max_samples=0.8, max_features=0.8,random_state=301)
# Fit model
model.fit(X_train, y_train)

BaggingRegressor(base_estimator=Lasso(alpha=0.1, max_iter=10000, positive=True,
                                      precompute=True, selection='random'),
                 max_features=0.8, max_samples=0.8, random_state=301)

In [41]:
# Evaluate model
train_preds = model.predict(X_train)
train_preds = np.maximum(train_preds, 0) # Don't predict negative cases
print('Train MAE:', mae(train_preds, y_train))

test_preds = model.predict(X_geo_test)
test_preds = np.maximum(test_preds, 0) # Don't predict negative cases
print('Test MAE:', mae(test_preds, y_geo_test))
df = pd.DataFrame({'test_preds': test_preds, 'y_test': y_geo_test.flatten()})
df

Train MAE: 253.63810283302087
Test MAE: 779.8209043584362


,test_preds,y_test
0,4061.606433,4797.0
1,4274.255542,5401.0
2,4695.699068,5388.0
3,4731.367823,3649.0
4,4096.057014,3490.0
5,4167.929030,5088.0
6,4885.276262,5294.0
7,5117.372392,5310.0
8,5346.918475,5468.0
9,5249.938883,5973.0


In [23]:
print(X_geo_test[-1])
print(y_geo_test[-1])

[ 4.820e+03  4.797e+03  5.401e+03  5.388e+03  3.649e+03  3.490e+03
  5.088e+03  5.294e+03  5.310e+03  5.468e+03  5.973e+03  5.391e+03
  5.906e+03  4.881e+03 -2.000e+00 -3.000e+00 -2.000e+00 -4.000e+00
 -1.000e+00 -1.000e+00 -2.000e+00 -3.000e+00 -2.000e+00 -3.000e+00
 -2.000e+00 -3.000e+00 -2.000e+00 -3.000e+00 -2.000e+00 -4.000e+00
 -1.000e+00 -1.000e+00 -2.000e+00 -3.000e+00 -2.000e+00 -3.000e+00
 -2.000e+00 -3.000e+00 -2.000e+00 -3.000e+00 -2.000e+00 -4.000e+00
 -1.000e+00 -1.000e+00 -2.000e+00 -3.000e+00 -2.000e+00 -3.000e+00
 -2.000e+00 -3.000e+00 -2.000e+00 -3.000e+00 -2.000e+00 -4.000e+00
 -1.000e+00 -1.000e+00 -2.000e+00 -3.000e+00 -2.000e+00 -3.000e+00
 -2.000e+00 -3.000e+00 -2.000e+00 -3.000e+00 -2.000e+00 -4.000e+00
 -1.000e+00 -1.000e+00 -2.000e+00 -3.000e+00 -2.000e+00 -3.000e+00
 -2.000e+00 -3.000e+00 -2.000e+00 -3.000e+00 -2.000e+00 -4.000e+00
 -1.000e+00 -1.000e+00 -2.000e+00 -3.000e+00 -2.000e+00 -3.000e+00
 -2.000e+00 -3.000e+00 -2.000e+00 -3.000e+00 -2.000e+00 -4.000

In [20]:
# Inspect the learned feature coefficients for the model
# to see what features it's paying attention to.

# Give names to the features
x_col_names = []
for d in range(-nb_lookback_days, 0):
    x_col_names.append('Day ' + str(d) + ' ' + cases_col[0])
for d in range(-nb_lookback_days, 1):
    for col_name in npi_cols:
        x_col_names.append('Day ' + str(d) + ' ' + col_name)

# View non-zero coefficients
for (col, coeff) in zip(x_col_names, list(model.coef_)):
    if coeff != 0.:
        print(col, coeff)
print('Intercept', model.intercept_)

AttributeError: 'BaggingRegressor' object has no attribute 'coef_'

In [ ]:
# Save model to file
if not os.path.exists('models'):
    os.mkdir('models')
with open('models/model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

## Evaluation

Now that the predictor has been trained and saved, this section contains the functionality for evaluating it on sample evaluation data.

In [ ]:
# Reload the module to get the latest changes
import predict
from importlib import reload
reload(predict)
from predict import predict_df

In [ ]:
# 会碰到历史数据很少而不能满足模型需求的情况（冷启动）
%%time
preds_df = predict_df("2020-08-01", "2020-08-31", path_to_ips_file="../../../validation/data/2020-09-30_historical_ip.csv", verbose=True)

In [ ]:
# Check the predictions
preds_df.head()

# Validation
This is how the predictor is going to be called during the competition.  
!!! PLEASE DO NOT CHANGE THE API !!!

In [ ]:
!python predict.py -s 2020-08-01 -e 2020-08-04 -ip ../../../validation/data/2020-09-30_historical_ip.csv -o predictions/2020-08-01_2020-08-04.csv

In [ ]:
!head predictions/2020-08-01_2020-08-04.csv

# Test cases
We can generate a prediction file. Let's validate a few cases...

In [ ]:
import os
from covid_xprize.validation.predictor_validation import validate_submission

def validate(start_date, end_date, ip_file, output_file):
    # First, delete any potential old file
    try:
        os.remove(output_file)
    except OSError:
        pass
    
    # Then generate the prediction, calling the official API
    !python predict.py -s {start_date} -e {end_date} -ip {ip_file} -o {output_file}
    
    # And validate it
    errors = validate_submission(start_date, end_date, ip_file, output_file)
    if errors:
        for error in errors:
            print(error)
    else:
        print("All good!")

## 4 days, no gap
- All countries and regions
- Official number of cases is known up to start_date
- Intervention Plans are the official ones

In [ ]:
validate(start_date="2020-08-01",
         end_date="2020-08-04",
         ip_file="../../../validation/data/2020-09-30_historical_ip.csv",
         output_file="predictions/val_4_days.csv")

## 1 month in the future
- 2 countries only
- there's a gap between date of last known number of cases and start_date
- For future dates, Intervention Plans contains scenarios for which predictions are requested to answer the question: what will happen if we apply these plans?

In [ ]:
%%time
validate(start_date="2021-01-01",
         end_date="2021-01-31",
         ip_file="../../../validation/data/future_ip.csv",
         output_file="predictions/val_1_month_future.csv")

## 180 days, from a future date, all countries and regions
- Prediction start date is 1 week from now. (i.e. assuming submission date is 1 week from now)  
- Prediction end date is 6 months after start date.  
- Prediction is requested for all available countries and regions.  
- Intervention plan scenario: freeze last known intervention plans for each country and region.  

As the number of cases is not known yet between today and start date, but the model relies on them, the model has to predict them in order to use them.  
This test is the most demanding test. It should take less than 1 hour to generate the prediction file.

### Generate the scenario

In [ ]:
from datetime import datetime, timedelta

start_date = datetime.now() + timedelta(days=7)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date = start_date + timedelta(days=180)
end_date_str = end_date.strftime('%Y-%m-%d')
print(f"Start date: {start_date_str}")
print(f"End date: {end_date_str}")

In [ ]:
from covid_xprize.validation.scenario_generator import get_raw_data, generate_scenario, NPI_COLUMNS
DATA_FILE = 'data/OxCGRT_latest.csv'
latest_df = get_raw_data(DATA_FILE, latest=True)
scenario_df = generate_scenario(start_date_str, end_date_str, latest_df, countries=None, scenario="Freeze")
scenario_file = "predictions/180_days_future_scenario.csv"
scenario_df.to_csv(scenario_file, index=False)
print(f"Saved scenario to {scenario_file}")

### Check it

In [ ]:
%%time
validate(start_date=start_date_str,
         end_date=end_date_str,
         ip_file=scenario_file,
         output_file="predictions/val_6_month_future.csv")